In [1]:
#事前建立環境選python3.6
#安裝openpyxl、pandas(python裡面的Excel)
import openpyxl
import pandas as pd
import os
import datetime
#from IPython.display import display

In [2]:
excelname= "kd"
sheetname = "kd"

wb = openpyxl.load_workbook(str(excelname)+".xlsx")
ws = wb[str(sheetname)]

dataset = []

i = 1
while(ws.cell(row=i, column=1).value!=None):
    row = []
    j = 1
    while(j < 6):
        row.append(ws.cell(row=i, column=j).value)
        j = j +1
    dataset.append(row)
    i = i + 1
#pd.DataFrame(dataset).head(10)

In [3]:
trade=[]
trade =[[0 for x in range(17)] for y in range(len(dataset))]
trade[0][0] = "date"
trade[0][1] = "open"
trade[0][2] = "high"
trade[0][3] = "low"
trade[0][4] = "close"
trade[0][5] = "rsv"
trade[0][6] = "k"
trade[0][7] = "d"
trade[0][8] = "buy_day"
trade[0][9] = "cum_day"
trade[0][10] = "sell_day"
trade[0][11] = "number"
trade[0][12] = "value"
trade[0][13] = "period_return"
trade[0][14] = "closeprice"
trade[0][15] = "days held"
trade[0][16] = "Annualized Return"

In [4]:
from datetime import datetime
for i in range(1, len(trade)):
    for j in range(0, 5):
        if (j == 0):
            trade[i][j] = datetime.strptime(dataset[i][j], '%Y/%m/%d')
        elif (j == 4):
            trade[i][j] = dataset[i][j]
            trade[i][14] = dataset[i][j]
        else:
            trade[i][j] = dataset[i][j]
        j = j + 1
    i = i + 1

In [16]:
t = 9  #9天
k = 50
d = 50
trade[t][6] = k;
trade[t][7] = d;

while(t < len(trade)):
    low = []  #每次清空list
    high = []
    for i in range(0, 9):
        low.append(float(trade[t - i][3]))   #填充9天最低價
        high.append(float(trade[t - i][2]))  #填充9天最高價
    
    #RSV(t) = (今日收盤價 - 最近九日最低價)/ (最近九日最高價 - 最近九日最低價)*100
    trade[t][5] = (float(trade[t][4]) - min(low)) / (max(high)-min(low)) *100  #填入RSV
    
    t = t + 1
    
    #計算當天K值時必須前一天已經有參考值所以KD值計算放在這裡執行
    if(t < len(trade)): #因為會跨到隔天所以要先檢查會不會超出LIST範圍 
        k = trade[t-1][6] * 2/3 + trade[t][5] * 1/3  #K(t)= K(t-1)*2/3 + RSV(t)*1/3, t = 10, 11, ..., T
        d = trade[t-1][7] * 2/3 + trade[t][6] * 1/3  #D(t)= D(t-1)*2/3 + K(t)*1/3  , t = 10, 11, ..., T
        if (t<=10):
            print(trade[t-1][6] * 2/3 + trade[t][5] * 1/3)
        trade[t][6] = k
        trade[t][7] = d
        
        

52.47793618465713


In [17]:
# 買進-8  庫存-9  賣出-10
for i in range(9, len(trade)):
    
    if (trade[i][6] > trade[i][7]) and (trade[i-1][9] == 0):
        trade[i][8] = 1
        trade[i][9] = 1
                
    elif (trade[i][6] < trade[i][7]) and (trade[i-1][9] != 0):
        trade[i][9] = 0
        trade[i][10] = 1
       
    else:
        trade[i][9] = trade[i-1][9]
        

In [94]:
initial = 10000

for i in range(10, len(trade)):
    if(trade[i][8] == 1)and(trade[i][9]==1)and(trade[i][10] == 0): #買進時
        first_day = trade[i-1][0]
        trade[i][11] = initial/trade[i][14] #持股數
        trade[i][12] = trade[i][4] * trade[i][11] #現值
        trade[i][13] = round((trade[i][4] * trade[i][11] - initial)/initial, 4) #報酬
        trade[i][15] = (trade[i][0] - first_day).days #持有天數
       
        trade[i][16] = (1+trade[i][13])**(365/trade[i][15])-1
        
    elif(trade[i][8] == 0)and(trade[i][9]==0)and(trade[i][10] == 1): #賣出時
        trade[i][11] = 0
        trade[i][12] = trade[i-1][11] * trade[i][14]
        trade[i][13] = round((trade[i-1][11] * trade[i][14] - initial)/initial, 4)
        trade[i][15] = (trade[i][0] - first_day).days #持有天數
        trade[i][16] = (1+trade[i][13])**(365/trade[i][15])-1
        
    elif(trade[i][8] == 0)and(trade[i][9]==1)and(trade[i][10] == 0): #持有時
        trade[i][11] = trade[i-1][11]
        trade[i][12] = trade[i][11] * trade[i][14]
        trade[i][13] = round((trade[i][11] * trade[i][14] - initial)/initial, 4)
        trade[i][15] = (trade[i][0] - first_day).days #持有天數
        trade[i][16] = (1+trade[i][13])**(365/trade[i][15])-1
        
        
    else:
        trade[i][15] = (trade[i][0] - trade[i][0]).days

In [95]:
pd.DataFrame(trade)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,date,open,high,low,close,rsv,k,d,buy_day,cum_day,sell_day,number,value,period_return,closeprice,days held,Annualized Return
1,2003-06-30 00:00:00,37.1,37.4,36.92,37.08,0,0,0,0,0,0,0,0,0,37.08,0,0
2,2003-07-01 00:00:00,37.09,38.1,37.09,38.05,0,0,0,0,0,0,0,0,0,38.05,0,0
3,2003-07-02 00:00:00,38.17,38.82,38.1,38.69,0,0,0,0,0,0,0,0,0,38.69,0,0
4,2003-07-03 00:00:00,40.6,40.6,38.81,39,0,0,0,0,0,0,0,0,0,39,0,0
5,2003-07-04 00:00:00,39.1,39.26,38.75,39.26,0,0,0,0,0,0,0,0,0,39.26,0,0
6,2003-07-07 00:00:00,39.6,41,39.6,41,0,0,0,0,0,0,0,0,0,41,0,0
7,2003-07-08 00:00:00,42,42,40.7,41.19,0,0,0,0,0,0,0,0,0,41.19,0,0
8,2003-07-09 00:00:00,41.3,41.3,40.81,41.22,0,0,0,0,0,0,0,0,0,41.22,0,0
9,2003-07-10 00:00:00,41.2,41.2,40.05,40.05,61.6142,50,50,0,0,0,0,0,0,40.05,0,0
